In [2]:
import dfBasics
import pandas as pd

version_sla = 'v00000'
version     = version_sla + '/v00000'

In [3]:
sparkSession = dfBasics.getSparkSession()

In [6]:
senders = list(sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/v00000/v00000/senders/senders.parquet').toPandas()['CSENDERENDPOINTID'])

In [13]:
senders_files = list(sparkSession.read.text('hdfs://172.30.17.145:8020/user/admin/sla/v00000/v00000/senders/senders_files.txt').toPandas()['value'])

In [17]:
len(senders_files), len(senders)

(784, 5493)

In [39]:
pfall = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + tested_senders_files[0]).toPandas()
pfall

,CGLOBALMESSAGEID,CSTARTTIME,CENDTIME,CSTATUS,CSERVICE,CSENDERENDPOINTID,CSENDERPROTOCOL,CINBOUNDSIZE,CRECEIVERPROTOCOL,CRECEIVERENDPOINTID,CSLATAT,CMESSAGETAT2,CSLADELIVERYTIME,year,month,day,hour,minute
0,52229092-3eae-11ea-abd6-95efac1e100b,1579872901144,1579872910984,11,6,14,4,17414,0,1002,9673,9840,1579872910817,2020,1,24,14,35
1,65f67655-fb05-11e9-9cf9-6e9eac1e100b,1572433621557,1572433635066,11,6,14,4,14515,0,1905,13432,13509,1572433634989,2019,10,30,12,7
2,b9727682-3ddb-11ea-9ecd-0a89ac1e100c,1579782450664,1579782463121,11,6,14,4,14219,0,1905,12357,12457,1579782463021,2020,1,23,13,27
3,6650a243-fb1b-11e9-92f2-f5adac1e100c,1572443071076,1572443079157,11,6,14,4,27300,0,1002,7928,8081,1572443079004,2019,10,30,14,44
4,ffe24923-3d1c-11ea-9ecd-0a89ac1e100c,1579700534962,1579700546002,11,6,14,4,15034,0,979,10847,11040,1579700545809,2020,1,22,14,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7528,a818f562-9db9-11ea-aad1-6c31ac1e100c,1590323130293,1590323135951,11,6,14,4,18030,0,1002,5608,5658,1590323135901,2020,5,24,14,25
7529,a9dc86e3-5e62-11eb-887f-be30ac1e100d,1611506341453,1611506345376,11,6,14,4,23546,0,1905,3894,3923,1611506345347,2021,1,24,17,39
7530,48d17f23-ad66-11ea-aad1-6c31ac1e100c,1592046540818,1592046546461,11,6,14,4,332828,0,1002,5588,5643,1592046546406,2020,6,13,13,9
7531,36ccead4-ad66-11ea-b980-22fcac1e100b,1592046510589,1592046515655,11,6,14,4,228284,0,1002,5018,5066,1592046515607,2020,6,13,13,8


In [43]:
pd.unique(pfall['CSERVICE'])

array([ 6, -1], dtype=int32)

In [32]:
skip_senders_files = []
for file in senders_files:
    try:
        pfall = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + file).toPandas()
        if -1 in pd.unique(pfall['CSENDERENDPOINTID']) or -1 in pd.unique(pfall['CRECEIVERENDPOINTID']):
            skip_senders_files.append(file)
    except Exception as exception:     
        skip_senders_files.append(file)

In [34]:
len(skip_senders_files)

596

In [38]:
tested_senders_files = [file for file in senders_files if file not in skip_senders_files]

In [2]:
import encoder
import numpy as np

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# restore np.load for future normal usage
#np.load = np_load_old

def inverse_transform(value,column=None, npy='/home/jovyan/work/npy'):
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    return _encoder.inverse_transform(value)  

In [3]:

column = 'CSERVICE'
npy='/home/jovyan/work/npy'
_encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
_encoder.classes_ = np.load(npy + '/' + column + '.npy')

In [6]:


  

inverse_transform(6,column='CSERVICE')

array('MaKo', dtype=object)